<a class="anchor" id="nutshell"></a>
<h2 style="font-family:'Verdana',sans-serif; color:#1D7874;">1. Alarm Feature for L2RPN ICAPS Competition</h2>

**Why an Alarm Feature?**
The main motivation of this competition is developing a robust reinforcement learning agent which a) can operate a power grid avoiding potential black out situations and b) capable of generating alarms to inform the human operators that the stiuation is out of its control. The environment considered is an uncertain indeed and also adversarial attacks are presenr to oppose the agent in achieving its goal.

Apart from providing timely corrective actions, it is intended that a good agent can raise an alarm at the appropriate instant. By 'appropriate instant' we mean that it should not be too far or too close from actual game-over (black out) conditions. If the agent generates an alarm close to potential failure, the remaining time may not be sufficient enough for the human operators to take decision and avert the failure. Also, it is not good to send spurious/false alarm to the operators when there is no threat to the grid operations. There may happen some other condition when an agent raises an alarm considering a failure is inevitable, but due to uncertainties of the environments the acutal failure is not occured. Hence, it is always a challenging task to design an agent which simultaneously make corrective actions and raise alarms in an uncertain environment. It is also important to note that the agent has a limited budget for this alarm feature, that means the no. of alarm can be raised is limited in a fixed duration. For more details, please visit Grid2op documentation.

In [ ]:
import numpy as np
import pandas as pd

import grid2op
print(grid2op.__version__)

# Backend class to use
try:
    from lightsim2grid.LightSimBackend import LightSimBackend
    BACKEND = LightSimBackend
except ModuleNotFoundError:
    from grid2op.Backend import PandaPowerBackend
    BACKEND = PandaPowerBackend

from grid2op.Runner import Runner
from grid2op.Parameters import Parameters

from grid2op.Agent.BaseAgent import BaseAgent
import numpy as np
import re

<a class="anchor" id="nutshell"></a>
<h2 style="font-family:'Verdana',sans-serif; color:#1D7874;">2. Use the latest Grid2op version and select the correct environment path.</h2>
<p style="font-family:'Verdana','sans-serif'; color:#393D3F; text-align:justify; font-size:14px;">
In the new version of Grid2op a new reward class related to alarm feature is provided.

In [ ]:


from grid2op.Reward import AlarmReward
env = grid2op.make("l2rpn_icaps_2021_small",backend=BACKEND()
                   ,reward_class=AlarmReward)



## Take a quick look at the grid

In [ ]:
from grid2op.PlotGrid import PlotMatplot
obs = env.reset()
plot_helper = PlotMatplot(env.observation_space, width=1920,height=1080, line_id=False)
_ = plot_helper.plot_layout()

<p style="font-family:'Verdana','sans-serif'; color:#393D3F; text-align:justify; font-size:14px;">
    This powergrid is the same one that the one used for the wcci competitions, except that we modified the names to match their original denomination.
    <br>
    It is made of 36 substations, with 22 generators and 29 loads and 59 lines.
    <br>
    It has the particularity to be a "subgrid" of the larger 118 grid as shown here :
    <img src="utils/img/R2_full_grid.png">
    The main consequence is that, when we generated the data, some energy was transfered from this area into the other. This is why you will notice some objects that we modeled as "loads" but those are in fact interconnection powerline. You can recognize them because they have "interco" in their names (see cell bellow). There are no major difference, for this competition between a regular loads or an interconnection powerline. The only difference is that sometimes interconnection powerlines can have negative values (regular loads always have positive value). We highlighed in thick dark green the interconnection powerline.
    <br><br>
    In the two cells bellow are printed first all the loads that represent interconnection powerlines (that can be either positive or negative) and then the normal loads (that can only be positive).
</p>

In [ ]:
[el for el in env.name_load if re.match(".*interco.*", el) is not None]

In [ ]:
[el for el in env.name_load if re.match(".*load.*", el) is not None]

<a class="anchor" id="nutshell"></a>
<h2 style="font-family:'Verdana',sans-serif; color:#1D7874;">3. A DoNothing_Attention_Agent</h2>

<p style="font-family:'Verdana','sans-serif'; color:#393D3F; text-align:justify; font-size:14px;">
This is an example of DoNothing_Attention_Agent, which can be made using the BaseAgent class of grid2op. The main purpose of this agent is only to genrate alarm depending upon the pre-defined criteria. 
<p style="font-family:'Verdana','sans-serif'; color:#393D3F; text-align:justify; font-size:14px;">
As per this example, the agent will do no action at all time, but can genrate regional/zonal alarm when an adverse condition is met. Hence, the expected output of this agent is a region/zone ID. This means, when the agent finds any trouble to operate the power grid an alarm indicating the region/zone ID "x" will be sent to the operator.  

In [ ]:
class DoNothing_Attention_Agent(BaseAgent):
    """
    This is the most basic BaseAgent. It is purely passive, and does absolutely nothing.
    As opposed to most reinforcement learning environments, in grid2op, doing nothing is often
    the best solution.
    """

    def __init__(self, action_space, alarms_lines_area):
        BaseAgent.__init__(self, action_space)
        self.alarms_lines_area = alarms_lines_area
        self.alarms_area_names = env.alarms_area_names

    def act(self, observation, reward, done=False):
        """
        As better explained in the document of :func:`grid2op.BaseAction.update` or
        :func:`grid2op.BaseAction.ActionSpace.__call__`.
        The preferred way to make an object of type action is to call :func:`grid2op.BaseAction.ActionSpace.__call__`
        with the dictionary representing the action. In this case, the action is "do nothing" and it is represented by
        the empty dictionary.
        Parameters
        ----------
        observation: :class:`grid2op.Observation.Observation`
            The current observation of the :class:`grid2op.Environment.Environment`
        reward: ``float``
            The current reward. This is the reward obtained by the previous action
        done: ``bool``
            Whether the episode has ended or not. Used to maintain gym compatibility
        Returns
        -------
        res: :class:`grid2op.Action.Action`
            The action chosen by the bot / controller / agent.
        """
        res = self.action_space({})
        if (np.max(observation.rho) >= 1):
            zones_alert = self.get_region_alert(observation)
            print(zones_alert)
            res = self.action_space({"raise_alarm": zones_alert})
            print(res)
        # print(res)
        return res

    def get_region_alert(self, observation):
        # extract the zones they belong too
        zones_these_lines = set()
        zone_for_each_lines = self.alarms_lines_area

        lines_overloaded = np.where(observation.rho >= 1)[0].tolist()  # obs.rho>0.6
        #print(lines_overloaded)
        for line_id in lines_overloaded:
            line_name = observation.name_line[line_id]
            for zone_name in zone_for_each_lines[line_name]:
                zones_these_lines.add(zone_name)

        zones_these_lines = list(zones_these_lines)
        zones_ids_these_lines = [self.alarms_area_names.index(zone) for zone in zones_these_lines]
        return zones_ids_these_lines


<a class="anchor" id="nutshell"></a>
<h2 style="font-family:'Verdana',sans-serif; color:#1D7874;"> 4. This is a simple example showing "how one can define an action of raising alarm"</h2>

In [ ]:
act_with_alarm = env.action_space({"raise_alarm": [2]})
print(act_with_alarm)

<a class="anchor" id="nutshell"></a>
<h2 style="font-family:'Verdana',sans-serif; color:#1D7874;"> 5. This is a test run of Do_Nothing_Attention Agent:</h2>

<p style="font-family:'Verdana','sans-serif'; color:#393D3F; text-align:justify; font-size:14px;">
The agent is taking no actions but generating alarms for region/zone ID -2 before game over.


In [ ]:
agent=DoNothing_Attention_Agent(env.action_space,env.alarms_lines_area)

runner = Runner(**env.get_params_for_runner(),
                agentClass=None,
                agentInstance=agent
                )


id_chonic=1
res_episode = runner.run_one_episode(detailed_output=True,indx=id_chonic,
                env_seed=1572415299,path_save='res_alert',
                agent_seed=1708891582)

name_chron, cum_reward, nb_timestep, episode_data = res_episode

print('time_since_last_alarm is: '+str(episode_data.observations[nb_timestep-2].time_since_last_alarm[0]+2))
print('Attention reward is: '+str(cum_reward))


<a class="anchor" id="nutshell"></a>
<h2 style="font-family:'Verdana',sans-serif; color:#1D7874;"> 6. One can check the relevant items in the observation related to alarm feature</h2>

In [ ]:
#look at what is in the observation
last_alarms = []
for obs in episode_data.observations[1:nb_timestep]:
    last_alarms.append(obs.time_since_last_alarm[0])
alarm_legal = [int(last_alarm == 0) for last_alarm in last_alarms]
df_le = pd.DataFrame({'alarm_legal': alarm_legal})
alarm_timesteps = np.where(df_le == 1)[0].tolist()

print('survival time is: '+str(nb_timestep))
print('alarm timesteps in observations are:'+str(alarm_timesteps))

In [ ]:
df_le.plot()

In [ ]:
obs = env.reset() # or any other information

obs.is_alarm_illegal
obs.time_since_last_alarm
obs.last_alarm
obs.attention_budget
obs.was_alarm_used_after_game_over

<a class="anchor" id="nutshell"></a>
<h2 style="font-family:'Verdana',sans-serif; color:#1D7874;"> 7. Zones/Regions for ICAPS track</h2>

<p style="font-family:'Verdana','sans-serif'; color:#393D3F; text-align:justify; font-size:14px;">
In this competition, the same grid of Neurips Track-1 is used, and the grid is divided into 3 zones- 1. East , 2. West and 3. Middle for controlling the network in a more organized way. This also helps to implement the "trust/alarm" feature of the agent. It also helps the agent to communicate with the human operators using the zonal/regional ID.

<img src="utils/img/grid_zone_alarms.png" width="800">

In [ ]:
from grid2op.PlotGrid import PlotMatplot


plot_helper = PlotMatplot(env.observation_space, 
                          sub_radius=14, 
                          load_radius=10, 
                          gen_radius=10,
                          width=950,
                          height=600,
                        )

In [ ]:
import json
import os

zone="east"
zone_id=env.alarms_area_names.index(zone)

print("now ploting lines in zone "+zone)
lines_east=env.alarms_area_lines[zone_id]
line_vals=[1 if l in lines_east else 0 for l in env.name_line]
plot_helper.plot_info(line_values=line_vals,coloring="line")